### Making the function of `np.linalg` and analyzing
> I'll study about the Theorems that i studied in 'Linear Algebra' class 

In [9]:
import numpy as np

#### Random Matrix

In [10]:
# Making the random matrix within 1 and 2 dimension

def new_m(m, n, min=-100, max=101):
    return np.random.randint(min, max, (m, n), dtype='int16')

### Function; checking the matrix whether it is a triangular or symmetric, and making a diagoanl
> Only based on it's definition, not on other theorems like determinant

#### Upper triangular

In [11]:
# Upper triangular: for all 0 <= i, j < n, i > j ; a[i][j] == 0

def ut(arr):
    if arr.shape == (1,):
        return True
    flag = False        # flag를 통해 행렬 내부에 삼각행렬의 정의에 위반되는 원소가 존재하는지 찾는다.
    n, _ = arr.shape    # Upper triangular는 n x n 행렬에서 정의된다.

    for j in range(n):
        if flag:
            break       # 반복문에서 flag의 값이 True로 변했다면 더이상 조건문을 돌릴 필요가 없다.
        for i in range(j + 1, n):
            if arr[i][j] != 0:      # arr의 i행 j열 원소 ; (i, j)th entry of a matrix, arr
                flag = True         # Upper triangular의 정의 조건에 위배될 때 flag = True
    if flag:
        return False    
    return True

In [12]:
test_sample = np.array([[1, 2, 3], [0, 2, 3], [0, 0, 1]])
print(ut(test_sample))

True


Lower Triangular의 함수도 같은 방식으로 선언된다.

#### Symmetric

In [31]:
# Symmetric: for all 0 <= i, j < n ; a[i][j] == a[j][i] ; which is a transpose of given matrix

def sym(arr):
    if arr.shape == (1,):
        return True                     # shape이 (1,)인 행렬(1 x 1)은 원소와 관계없이 정방행렬이다.
    n, m = arr.shape
    if n != m:
        return False                    # 정방행렬이 아닌 경우 제거                  
    
    for i in range(n):
        for j in range(i + 1, n):       # 모든 (i, j)를 둘러볼 필요가 없다.
            if arr[i][j] != arr[j][i]:
                return False            # flag보다 더 효율적인 방식이다.
    return True                         # for문이 중간에 끊기지 않음 = 주어진 행렬이 symmetric이다.

arr = np.array([[1, 2], [2, 4]])
print(sym(arr))


True


#### Diagonal

In [14]:
# np.linalg.diagonal()가 존재한다.

arr = new_m(3, 3)
print(arr)
print(np.linalg.diagonal(arr))

[[ 53  76 -56]
 [-78  69 -79]
 [ 18 -29  72]]
[53 69 72]


In [32]:
# diagonal을 구하는 함수를 만들어 보았다.

def dia(mat):      
    if type(mat) != np.ndarray:         # mat의 클래스를 확인하고 예외처리
        raise TypeError("please give the <class 'numpy.dnarray'>") 
    res = []
    
    for i in range(min(mat.shape)):     # 주어진 함수가 n x n이 아닌 경우를 고려하여 반복수 조정정
        res.append(mat[i][i])           # diagonal의 정의에 따라 행과 열의 인덱스가 같은 원소 추출

    return np.array(res)            # numpy.ndarray 클래스로 출력한다.

In [16]:
# 클래스 예외처리 시험

print(dia(np.array([[1, 2], [3, 4]])))
# print(dia([1, 2]))

[1 4]


> 성능 비교

In [33]:
# 시간 측정을 위한 test_sample 추출

test_sample = new_m(100, 100)
print(test_sample)

[[-59  93  26 ...  99  77 -16]
 [-34  89 -92 ... -17 100   4]
 [ 77  90 -57 ...  93 -48  40]
 ...
 [-84  64   6 ...  76 -21 -68]
 [ 72  61  25 ...  22 -58 -90]
 [ 65  30  18 ...  77 -45 -14]]


In [34]:
%%timeit

dia(test_sample)

10.8 μs ± 43.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [35]:
%%timeit

np.linalg.diagonal(test_sample)

327 ns ± 0.925 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


`dia()` 와 `np.linalg.diagonal()`의 성능 차이(시간)가 대략 30배이다. 

#### Optimization using ChatGPT

>#### Suggestion <br>

1. `type(mat) != np.ndarray` &ensp; `type` 비교는 서브클래스 놓침 &ensp; `isinstance(mat, np.ndarray)` 사용<br>
2. `for + append` &ensp; 느린 루프 방식 &ensp;	벡터화된 `NumPy` 함수 사용<br>
3. `np.array(res)`	&ensp; 한 번 리스트 → 배열 &ensp; 처음부터 `NumPy` 배열 방식으로 계산


In [20]:
def dia_optimized(mat):
    if not isinstance(mat, np.ndarray):
        raise TypeError("Input must be a numpy.ndarray")

    m, n = mat.shape
    idx = np.arange(min(m, n))      # 인덱스 배열 생성
    return mat[idx, idx]            # Numpy의 브로드캐스팅 인덱싱

In [21]:
# dia_optimized 오류 확인
dia_optimized(test_sample) == np.linalg.diagonal(test_sample)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

> 성능 비교

In [36]:
%%timeit

dia_optimized(test_sample)

857 ns ± 4.52 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [44]:
import inspect

print(inspect.getsource(np.linalg.diagonal))

@array_function_dispatch(_diagonal_dispatcher)
def diagonal(x, /, *, offset=0):
    """
    Returns specified diagonals of a matrix (or a stack of matrices) ``x``.

    This function is Array API compatible, contrary to
    :py:func:`numpy.diagonal`, the matrix is assumed
    to be defined by the last two dimensions.

    Parameters
    ----------
    x : (...,M,N) array_like
        Input array having shape (..., M, N) and whose innermost two
        dimensions form MxN matrices.
    offset : int, optional
        Offset specifying the off-diagonal relative to the main diagonal,
        where::

            * offset = 0: the main diagonal.
            * offset > 0: off-diagonal above the main diagonal.
            * offset < 0: off-diagonal below the main diagonal.

    Returns
    -------
    out : (...,min(N,M)) ndarray
        An array containing the diagonals and whose shape is determined by
        removing the last two dimensions and appending a dimension equal to
        the si